In [4]:
from time import sleep
import requests
from pandas import DataFrame
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

In [3]:
path = 'C:/Users/USER/Desktop/종민/YBIGTA/2021-2 교육세션/0805 크롤링/chromedriver.exe'
driver = webdriver.Chrome(path)

url = 'https://www.wine21.com/13_search/monthly_list.html'
driver.get(url)
driver.implicitly_wait(1)

* 오른쪽 상단에 년도와 월을 선택할 수 있는 드롭다운 메뉴가 있다.

* Selenium에서 이를 선택하려면 Select를 사용해야한다.

* 테이스팅노트 탭에는 메이커스노트와 테이스팅노트가 따로 있다.

* 각 노트는 없을 수도 있고, 테이스팅노트의 경우 5개 이상일 때 더보기 탭이 추가된다.

In [67]:
df = DataFrame(columns=['한글 이름', '영문 이름', '가격', '종류', '나라', '도시',
                        '아로마', '음식매칭', '전문가 평점', '소비자 평점',
                        '스타일', '등급', '알코올', '음용온도', '당도', '산도',
                        '바디', '타닌', '메이커스 노트', '테이스팅 노트'])

year = '#shYear'
month = '#shMonth'

name_ko = 'body > section > div.inner > div.clear > div.wine-top-right > dl > dt'
name_en = 'body > section > div.inner > div.clear > div.wine-top-right > dl > dd'
price = 'body > section > div.inner > div.clear > div.wine-top-right > p.wine-price > strong'

kinds = 'body > section > div.inner > div.clear > div.wine-top-right > div.bagde-box > p > span.bagde-item.bagde-yellow-fill'
country = 'body > section > div.inner > div.clear > div.wine-top-right > div.bagde-box > p > span:nth-child(2)'
city = 'body > section > div.inner > div.clear > div.wine-top-right > div.bagde-box > p > span:nth-child(3)'

aroma = 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-top-right-inner > div:nth-child(1) > div.wine-matching-list.swiper-container.swiper-container-initialized.swiper-container-horizontal > ul'
food = 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-top-right-inner > div:nth-child(2) > div.wine-matching-list.swiper-container.swiper-container-initialized.swiper-container-horizontal > ul'

score_expert = 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-top-right-inner > div.score > div > dl.btn-pop-wine-03 > dd'
score_customer = 'body > section > div.inner > div.clear > div.wine-top-right > div.wine-top-right-inner > div.score > div > dl:nth-child(2) > dd > span'

style = '#detail > div > div > dl:nth-child(4) > dd'
grade = '#detail > div > div > dl:nth-child(5) > dd'
alcohol = '#detail > div > div > dl:nth-child(6) > dd'
temperature = '#detail > div > div > dl:nth-child(7) > dd'

more = '#tasting > div > div.makersnote-list2 > div.pagination-box > a'
maker = '#tasting > div > div.makersnote-list1 > div.board-list.board-list-makers > div > div.makers-item-tt > div'
taste = '#tasting > div > div.makersnote-list2'

css_selectors = [name_ko, name_en, price, kinds, country, city, aroma, food,
                score_expert, score_customer, style, grade, alcohol, temperature]

In [71]:
for y in range(2012, 2022):
    
    # 년도 고르기
    select = Select(driver.find_element_by_css_selector(year))
    select.select_by_value(str(y))
    
    for m in range(1, 13):
        
        # 월 고르기
        select = Select(driver.find_element_by_css_selector(month))
        select.select_by_value(str(m).zfill(2))
        
        i = 2
        while True:
            css = f'#frmSubView > section > div > div.wrap-board-left > div.board-list.board-list-wine > ul > li:nth-child({i}) > div.txt-area > div.cnt-header > h3 > a'
            try:
                search = driver.find_element_by_css_selector(css)
            except:
                break
            
            # 와인 상세 정보 클릭
            search.send_keys(Keys.RETURN)
            
            idx = len(df)
            
            # 와인 정보
            for css_selector, column in zip(css_selectors, df.columns[:-4]):
                try:
                    search = driver.find_element_by_css_selector(css_selector)
                    df.loc[idx, column] = search.text
                except:
                    pass
            
            # 당도, 산도, 바디, 타닌
            for p, column in enumerate(['당도', '산도', '바디', '타닌']):
                for q in range(1, 6):
                    css_selector = f'body > section > div.inner > div.clear > div.wine-top-right > div.wine-components > ul > li:nth-child({p + 1}) > div > a:nth-child({q})'
                    search = driver.find_element_by_css_selector(css_selector)
                    if search.get_attribute('class') != 'on':
                        df.loc[idx, column] = q - 1
                        break
                else:
                    df.loc[idx, column] = 5
            
            # 더보기 있으면 클릭
            flag = False
            try:
                search = driver.find_element_by_css_selector(more)
                search.send_keys(Keys.RETURN)
                flag = True
            except:
                pass
            
            try:
                search = driver.find_element_by_css_selector(maker)
                df.loc[idx, '메이커스 노트'] = search.text
            except:
                pass

            try:
                search = driver.find_element_by_css_selector(taste)
                df.loc[idx, '테이스팅 노트'] = search.text
            except:
                pass
            
            # 더보기를 눌렀다면 뒤로 가기를 두 번 해야함
            if flag:
                driver.back()
            
            driver.back()
            sleep(1)
            
            i += 1

In [72]:
df.to_csv('wine_of_the_month.csv')